In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Set Up Environment

In [ ]:
network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20210122074935.pkl'

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://52.53.175.151:9000/",       "nid": 3}}

env = connections[network]

In [ ]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [ ]:
contracts = {}

# with open(contracts_file_custom, 'rb') as f:
#     contracts = pkl.load(f)

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': ''},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': ''},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': ''},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': ''},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': ''},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': ''},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': ''},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': ''},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': ''},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': ''},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': ''},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': ''}}

In [ ]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")

In [ ]:
# Balanced test wallet
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)

### Send ICX to Balanced test wallet

In [ ]:
test_wallet = KeyWallet.load("./keystores/keystore_test1.json", "test1_Account")
transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(btest_wallet.get_address())\
    .value(1000000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, test_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

In [710]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
# contracts['dividends'] = {'zip': 'core_contracts/dividends.zip', 'SCORE': 'cxceadb6419b92f7170b38b8328dbd8deefb4a1cdb'}
deploy = ['loans', 'staking', 'dividends', 'rewards', 'reserve_fund', 'governance', 'dummy_oracle', 'sicx', 'dex', 'icd', 'bal', 'bwt']
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'

contracts

{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx0aa51835382471ff9c45bc6bebcb5d01711304fc'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx1d0f60dd70269836acc6c774642841d3fd29d29f'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx494148736fd3f3833590017053954b0c7729c356'},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': 'cxfdab420828e24a526da0b5d3babc5879f1d2152b'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx00f9315505a298e30dee40cf70a7b9e9a386b87b'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cxc44271fe5b7ef9673df45acc80f7d118f09a5054'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx92ade83fff6e7e07111385752c26cb7b5059e779'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx9a0795a01e955611211e2fa5b921eea2b3801033'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx36f2e2c78f961727fb05230850329ac79c9be76a'},
 'icd': {'zip':

In [ ]:
# save contracts to file
# now = datetime.utcnow()
# filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
# print(f'saving contracts list as {filename}')
# with open(filename, 'wb') as f:
#     pkl.dump(contracts, f)

In [ ]:
def get_scores_json(contracts):
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

In [701]:
print(get_scores_json(contracts))

{"loans": "cx0aa51835382471ff9c45bc6bebcb5d01711304fc", "staking": "cx1d0f60dd70269836acc6c774642841d3fd29d29f", "dividends": "cx494148736fd3f3833590017053954b0c7729c356", "reserve_fund": "cxfdab420828e24a526da0b5d3babc5879f1d2152b", "rewards": "cx00f9315505a298e30dee40cf70a7b9e9a386b87b", "dex": "cxc44271fe5b7ef9673df45acc80f7d118f09a5054", "governance": "cx92ade83fff6e7e07111385752c26cb7b5059e779", "dummy_oracle": "cx9a0795a01e955611211e2fa5b921eea2b3801033", "sicx": "cx36f2e2c78f961727fb05230850329ac79c9be76a", "icd": "cxddca542df30807927af19c4211341aa70995b28f", "bal": "cxd854e3848bcee9f34b858de826d7c504f8c75024", "bwt": "cxe044e897fac30c3c6b228c08117c97ecb545a27f"}


## Deploy All SCOREs

In [ ]:
step_limit = 4000100000

for score, loc in contracts.items():
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

#     step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
    print(score)
    print(res)

contracts

### Deploy a single SCORE

In [ ]:
# Deploy a single SCORE
contract = 'governance'
update = contracts[contract]['SCORE']
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

step_limit = 10000000000


signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
contracts[contract]['SCORE'] = res.get('scoreAddress', '')
res

## Update a SCORE

In [711]:
contract = 'dividends'
update = contracts[contract]['SCORE']
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

# step_limit = icon_service.estimate_step(deploy_transaction) + 100000
step_limit = 3000000000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


{'txHash': '0xde4a2db1c79147064b2e305825bfcc58c0295d2c4e9154a09d787674ff2a4357',
 'blockHeight': 793650,
 'blockHash': '0x26dd46e7ff933838fb695b5174b434d5c1caccf33a6e7e0f8cc061668d75b65e',
 'txIndex': 1,
 'to': 'cx494148736fd3f3833590017053954b0c7729c356',
 'scoreAddress': 'cx494148736fd3f3833590017053954b0c7729c356',
 'stepUsed': 1836834400,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1836834400,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Set parameters

In [699]:
addresses = {'loans': contracts['loans']['SCORE'],
             'dex': contracts['dex']['SCORE'],
             'staking': contracts['staking']['SCORE'],
             'rewards': contracts['rewards']['SCORE'],
             'reserve_fund': contracts['reserve_fund']['SCORE'],
             'dividends': contracts['dividends']['SCORE'],
             'oracle': contracts['dummy_oracle']['SCORE'],
             'sicx': contracts['sicx']['SCORE'],
             'icd': contracts['icd']['SCORE'],
             'baln': contracts['bal']['SCORE'],
             'bwt': contracts['bwt']['SCORE'],
            }

settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'sicx', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'icd', 'method': 'setAdmin', 'params':{'_admin': contracts['loans']['SCORE']}},
            {'contract': 'icd', 'method': 'setOracle', 'params':{'_address': contracts['dummy_oracle']['SCORE'], '_name': 'Dummy'}},
            {'contract': 'bal', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'bal', 'method': 'mint', 'params':{'_amount': 10 * ICX}},
            {'contract': 'bwt', 'method': 'setAdmin', 'params':{'_admin': contracts['governance']['SCORE']}},
            {'contract': 'staking', 'method': 'setSicxAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'setLoansScore', 'params':{'_address': contracts['loans']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'setBalnToken', 'params':{'_address': contracts['bal']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'setSicxToken', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'bal', 'method': 'mintTo', 'params':{'_account': contracts['reserve_fund']['SCORE'], '_amount': 1000 * ICX}},
            {'contract': 'dividends', 'method': 'setLoansScore', 'params':{'_address': contracts['loans']['SCORE']}},
            {'contract': 'rewards', 'method': 'setBalnAddress', 'params':{'_address': contracts['bal']['SCORE']}},
            {'contract': 'rewards', 'method': 'setBatchSize', 'params':{'_batch_size': 10}},
            {'contract': 'rewards', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'rewards', 'method': 'addNewDataSource', 'params':{'_data_source_name': 'Loans', '_contract_address': contracts['loans']['SCORE']}},
            {'contract': 'rewards', 'method': 'addNewDataSource', 'params':{'_data_source_name': 'SICXICX', '_contract_address': contracts['dex']['SCORE']}},
            {'contract': 'dex', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'dex', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'dex', 'method': 'setRewards', 'params':{'_address': contracts['rewards']['SCORE']}},
            {'contract': 'dex', 'method': 'setStaking', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'dex', 'method': 'setSicx', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'dex', 'method': 'setDividends', 'params':{'_address': contracts['dividends']['SCORE']}},
            {'contract': 'loans', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'loans', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'loans', 'method': 'setDividends', 'params':{'_address': contracts['dividends']['SCORE']}},
            {'contract': 'loans', 'method': 'setReserve', 'params':{'_address': contracts['reserve_fund']['SCORE']}},
            {'contract': 'loans', 'method': 'setRewards', 'params':{'_address': contracts['rewards']['SCORE']}},
            {'contract': 'loans', 'method': 'addAsset', 'params':{'_token_address': contracts['sicx']['SCORE'], 'is_active': 1, 'is_collateral': 1}},
            {'contract': 'loans', 'method': 'addAsset', 'params':{'_token_address': contracts['icd']['SCORE'], 'is_active': 1, 'is_collateral': 0}},
            {'contract': 'loans', 'method': 'addAsset', 'params':{'_token_address': contracts['bal']['SCORE'], 'is_active': 0, 'is_collateral': 1}},
            {'contract': 'governance', 'method': 'setAddresses', 'params':{'_addresses': addresses}},
            {'contract': 'governance', 'method': 'launchBalanced', 'params':{}}]

In [700]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}




Calling setAdmin, with parameters {'_admin': 'cx1d0f60dd70269836acc6c774642841d3fd29d29f'} on the sicx contract.
Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


...

{'txHash': '0x89e1f0946246d16a06a777e7431cba7decb2510621dc9c257d639baceb0cc4ea', 'blockHeight': 793052, 'blockHash': '0xd528098614147d0676830b51e461b8993ba17f20d650fc777be116db6b144045', 'txIndex': 1, 'to': 'cx0aa51835382471ff9c45bc6bebcb5d01711304fc', 'stepUsed': 151800, 'stepPrice': 10000000000, 'cumulativeStepUsed': 151800, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'txHash': '0x7fc793b7ca645d73bd23ed263dddd7cb09cb695019799de6e8f56c292f209f99', 'blockHeight': 793054, 'blockHash': '0x380cfbe3e8d6c73a43c2b9528548fd5915ad1718a3b8421c277822df860d4822', 'txIndex': 1, 'to': 'cx92ade83fff6e7e07111385752c26cb7b5059e779', 'stepUsed': 271880, 'stepPrice': 10000000000, 'cumulativeStepUsed': 271880, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Run through read-only methods on all SCOREs

### A single read-only method

In [ ]:
params = {}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['staking']['SCORE'])\
                    .method('getSicxAddress')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result
# int(result, 0)

In [704]:
ro_methods = {"loans": ["name", "getAvailableAssets", "assetCount", "getCollateralTokens", "getTotalCollateral"],
              "staking": ["name", "getSicxAddress", "getTodayRate"],
              "dividends": ["name", "getBalances"],
              "governance": ["name", "getAddresses"],
              "sicx": ["name", "symbol", "decimals", "totalSupply", "getAdmin", "lastPriceInLoop"],
              "icd": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "bal": ["name", "symbol", "decimals", "totalSupply", "getAdmin"]
              }

In [705]:
results = {}
for score in ro_methods.keys():
    print()
    print(f'SCORE: {score}')
    print(f'Address: {contracts[score]["SCORE"]}')
    results[score] = {}
    for method in ro_methods[score]:
        call = CallBuilder().from_(wallet.get_address())\
                            .to(contracts[score]['SCORE'])\
                            .method(method)\
                            .build()
        result = icon_service.call(call)
        if method in ['totalSupply', 'decimals', 'get_total_collateral', 'get_rate']:
            result = int(result, 0)
            if method in ['totalSupply', 'get_total_collateral', 'get_rate']:
                result = result / ICX
        print(f'{method}: {result}')
        results[score][method] = result


SCORE: loans
Address: cx0aa51835382471ff9c45bc6bebcb5d01711304fc
name: BalancedLoans
getAvailableAssets: {'sICX': {'symbol': 'sICX', 'address': 'cx36f2e2c78f961727fb05230850329ac79c9be76a', 'peg': 'sICX', 'added': '0x5b9a367301bd2', 'is_collateral': '0x1', 'active': '0x1'}, 'ICD': {'symbol': 'ICD', 'address': 'cxddca542df30807927af19c4211341aa70995b28f', 'peg': 'USD', 'added': '0x5b9a3674ea478', 'is_collateral': '0x0', 'active': '0x1', 'bad_debt': '0xad78ebc5ac6200000', 'liquidation_pool': '0x2a2651bdae9c494800'}}
assetCount: 0x3
getCollateralTokens: {'sICX': 'cx36f2e2c78f961727fb05230850329ac79c9be76a', 'BALN': 'cxd854e3848bcee9f34b858de826d7c504f8c75024'}
getTotalCollateral: 0x57f8719fb38bde8000

SCORE: staking
Address: cx1d0f60dd70269836acc6c774642841d3fd29d29f
name: Staking


Raised JSONRPCException while returning the custom response. Error message: {'code': -32001, 'message': "'LoansInterface' object has no attribute 'getAssetTokens'"}


getSicxAddress: cx36f2e2c78f961727fb05230850329ac79c9be76a
getTodayRate: 0xde0b6b3a7640000

SCORE: dividends
Address: cx494148736fd3f3833590017053954b0c7729c356
name: Dividends


JSONRPCException: {'code': -32001, 'message': "'LoansInterface' object has no attribute 'getAssetTokens'"} (Json rpc error)

## Deposit ICX and Mint ICD Loan

In [ ]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"ICD\", \"_amount\": 40000000000000000000}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

### Just deposit collateral

In [ ]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(30 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

### Repay Some ICD

In [ ]:
data = "{\"method\": \"_repay_loan\", \"params\": {}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 5 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

### Withdraw Some Collateral

In [ ]:
params = {'_value': 5 * ICX}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("withdrawCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

### Test Liquidation

In [ ]:
# 1. Deposit collateral to fresh wallet

data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(782769 * ICX // 1000)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

In [ ]:
# 2. Check the account standing after depositing collateral - Should be Undetermined, and
#    holding '0x2a6f1a22364bbe8000' worth of sICX.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

#### 3. Update the loans SCORE to add a loan of ICD to the Balanced test wallet
Code in the on_update() method will mint and add the ICD. Adds an undercollateralized amount of ICD, 200 ICD.

In [ ]:
# 4. Check the new standing of the account. Should have added '0xad78ebc5ac6200000' ICD and have standing of 'Liquidate'.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
# 5. Liquidate the account position.

params = {'_owner': btest_wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("liquidate")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

In [ ]:
# 6. Check the standing of the account after liquidation. Should now have zero balance for sICX and ICD.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['icd']['SCORE'])\
                    .method('balanceOf')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
int(result, 0) // 10**18

### DEX SICX Test

Does the following:

1. Sends ICX to the DEX
2. Checks overall ICX balance
3. Checks withdrawal lock
4. Mints some SICX
5. Trades SICX against ICX

In [ ]:
transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXWithdrawLock')\
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
params = {'_to': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(10 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
data = "{\"method\": \"_swap_icx\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDataBatch')\
                    .params({'_name': 'SICXICX', '_limit': 10, '_snapshot_id': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDeposit')\
                    .params({'_user': wallet.get_address(), '_tokenAddress': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['icd']['SCORE'], '_baseValue': 1 * ICX, '_quoteValue': 1 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("permit")\
    .params({'_pid': 1, '_permission': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPrice')\
                    .params({'_pid': 1}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 1, '_token': contracts['sicx']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolId')\
                    .params({'_token1Address': contracts['sicx']['SCORE'], '_token2Address': contracts['icd']['SCORE'] }) \
                    .build()
result = icon_service.call(call)
print("pool w/ tokens: " + str(contracts['sicx']['SCORE']) + ' & ' + str(contracts['icd']['SCORE']))
print(result)

In [ ]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolBase')\
                    .params({'_pid': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolQuote')\
                    .params({'_pid': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':1, '_name': 'SICXICD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
to_token = contracts['sicx']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\"}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX / 10), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))
